# Step 1) Select trading strategy

This notebook requires that a container has been already deployed to ECR. Comment/uncomment the line in the following cell to select the trading strategy.

In [ ]:
algo_name='simple_sma'

# Step 2) Host trading strategy in SageMaker

In [ ]:
import sagemaker as sage
from sagemaker.model import * 

model_name='algo_'+algo_name

role = sage.get_execution_role()
sess = sage.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = ('{}.dkr.ecr.{}.amazonaws.com/'+model_name+':latest').format(account, region)

prefix='algo_'+algo_name
model_name_final=model_name.replace('_','-')+'-1'

# create a new Model object
m = Model(
    image=image,
    model_data='',
    role=role,
    name=model_name_final,
    sagemaker_session=sess
)
m.deploy(1, 'ml.t2.medium')

# Step 3) Create SQS queue for market data

This queue receives market data updates.

In [ ]:
!aws sqs create-queue --queue-name algo_market_data_queue.fifo --attributes FifoQueue=true,ContentBasedDeduplication=true,ReceiveMessageWaitTimeSeconds=5

# Step 4) Deploy Lambda function for market data

This lambda function is called from trading strategy to pull new market data.

In [ ]:
%%writefile lambda_function.py
import json
import boto3
import sys
import time

def lambda_handler(event, context):
    list=[]
    try:
        sqs=boto3.resource('sqs')
        queueIn=sqs.get_queue_by_name(QueueName='algo_market_data_queue.fifo')
        for message in queueIn.receive_messages():
            try:
                print("msg:%s" % message)
                item=json.loads(message.body)
                ret=json.dumps(item)
                list.append(item)
                print("ret:%s" % ret)
                message.delete()
                print("msg deleted")
                break
            except Exception as e:
                print("err:%s" % e)
    except:
        e = sys.exc_info()[0]
        print("err:%s" % e)
        time.sleep(5)
    return list

In [ ]:
!zip -g function.zip lambda_function.py
!aws lambda update-function-code --function-name algo_market_data --zip-file fileb://function.zip
!aws lambda update-function-configuration --function-name algo_market_data --timeout 10

# Step 5) Generate simulation data

In [ ]:
!pip install backtrader

In [ ]:
%run data_prep.py INTC

import pandas as pd
from model.algo_sim_feed import AlgoSimData

df = pd.read_csv("../1_Data/INTC.csv",infer_datetime_format=True, parse_dates=['dt'], index_col=['dt'])
histFile='local_test/test_dir/input/data/training/data.csv'

trainCount=int(len(df)*0.4)
dfTest = df.iloc[trainCount:]
dfTest.to_csv(histFile)

simData = AlgoSimData(histFile)
dfTest = simData.df

In [ ]:
%matplotlib notebook
dfTest[0].plot()

# Step 6) Run live simulation against hosted strategy

In [ ]:
import boto3
import time

sqs=boto3.resource('sqs')
queue=sqs.get_queue_by_name(QueueName='algo_market_data_queue.fifo')

def send_msg(m):
    global queue
    data=json.dumps(m)
    print("msg=%s" % data)
    try:
        response = queue.send_message(
            MessageBody=data,
            MessageGroupId='group1',
        )
        #print("ok:%s" % response)
    except Exception as e:
        print("err:%s" % e)
        
for idx,row in dfTest.iterrows():
    dt=row['index'].strftime("%Y-%m-%d")
    close=row[0]
    m={'date':dt, 'close':close}
    send_msg(m)
    time.sleep(1)